In [1]:
import boto3
import botocore
import logging
import sys
import json

In [2]:
s3 = boto3.resource('s3')

In [3]:
for bucket in s3.buckets.all():
    print(bucket.name)

ab3code
ab3demo-0a78222656a4-034186276030-ap-south-1-ingest
ab3demo-0a78222656a4-034186276030-ap-south-1-logs
ab3demo-0a78222656a4-034186276030-ap-south-1-proxy
ab3demo-0a78222656a4-034186276030-ap-south-1-web
ab3demo-0add957f0e7a-034186276030-ap-south-1-ingest
ab3demo-0add957f0e7a-034186276030-ap-south-1-logs
ab3demo-0add957f0e7a-034186276030-ap-south-1-proxy
ab3demo-0add957f0e7a-034186276030-ap-south-1-web
cloudtrail-awslogs-034186276030-nokas1gs-isengard-do-not-delete
do-not-delete-gatedgarden-audit-034186276030


In [18]:
client = boto3.client('s3')
BUCKET_NAME = 'ab3demo-0add957f0e7a-034186276030-ap-south-1-proxy'

media_data = []
for key in client.list_objects(Bucket=BUCKET_NAME, MaxKeys=10000)['Contents']:
    prefix = key['Key']
    if prefix.endswith(".mp4"):
        prefix_parts = prefix.split("/")
        v = {'id': prefix_parts[0], 'title': prefix_parts[-1][:-4].replace('"', '').replace("'", '')}
        media_data.append(v)

print(media_data)

[{'id': '3e025098-b828-38d1-609a-d334a5696631', 'title': 'Chennais Migrant Labourers Now Homeward Bound Due To Demonetisation'}, {'id': '475dfff8-b669-627b-997e-f060494a38e5', 'title': 'Arun Jaitley Talks About Demonetisation And Its Positive Effect'}, {'id': '562d2f91-d16e-83c6-451b-0c394347747c', 'title': 'Mukesh Ambani Lauds Demonetisation At World Economic Forum'}, {'id': '990b4946-5dff-d451-ce50-b8c2c0ec3bf3', 'title': 'Rupee Demonetisation- Mumbai Property Market Takes A Hit'}, {'id': 'ae728edc-40ea-ed4d-6e92-024e70e34175', 'title': 'The Queen visits New Delhi 1983'}]


In [19]:
for v in media_data:
    KEY = v['id'] + '/metadata/'
    
    s3.Bucket(BUCKET_NAME).download_file(KEY + 'sentiment/output.json', 'sentiment_output.json')
    s3.Bucket(BUCKET_NAME).download_file(KEY + 'keyphrase/output.json', 'keyphrase_output.json')
    
    with open('sentiment_output.json',) as f:
        v['sentiment'] = json.load(f)

    with open('keyphrase_output.json',) as f:
        v['keyphrase'] = json.load(f)

print(media_data)

[{'id': '3e025098-b828-38d1-609a-d334a5696631', 'title': 'Chennais Migrant Labourers Now Homeward Bound Due To Demonetisation', 'sentiment': [{'text': 'Neutral', 'confidence': 43.76, 'begin': 40, 'end': 57550}, {'text': 'Negative', 'confidence': 83.4, 'begin': 60040, 'end': 115650}, {'text': 'Negative', 'confidence': 73.8, 'begin': 116140, 'end': 153650}], 'keyphrase': [{'text': 'your news', 'confidence': 100, 'begin': 40, 'end': 3030}, {'text': 'the South', 'confidence': 99.96, 'begin': 40, 'end': 3030}, {'text': 'one segment', 'confidence': 99.95, 'begin': 3250, 'end': 7140}, {'text': 'the cash crunch', 'confidence': 99.83, 'begin': 7140, 'end': 8950}, {'text': 'right guard', 'confidence': 98.92, 'begin': 13340, 'end': 14370}, {'text': 'The migrant labourers', 'confidence': 100, 'begin': 14370, 'end': 16440}, {'text': 'a T m', 'confidence': 99.98, 'begin': 16440, 'end': 19790}, {'text': 'Taiwan', 'confidence': 100, 'begin': 16440, 'end': 19790}, {'text': 'the podium', 'confidence': 9

In [34]:
lines = []
tmp_set = set()
tmp_list = []

lines.append('g.addV("sentiment").property(id, "neutral")\n')
lines.append('g.addV("sentiment").property(id, "positive")\n')
lines.append('g.addV("sentiment").property(id, "negative")\n')
lines.append('g.addV("sentiment").property(id, "mixed")\n')

for post in media_data:
    lines.append('g.addV("video").property(id, "'+post['id']+'").property("title", "'+post['title']+'")\n')
        
    #TODO: Put logic to find the most suitable sentiment
    for sentiment in post['sentiment'][0:1]:
        # f.write('g.addV("sentiment").property(id, "'+sentiment['text'].lower()+'").next()\n')

        lines.append('g.addE("relates").from(g.V("'+post['id']+'")).to(g.V("'+sentiment['text'].lower()+'")).property("confidence", "'+str(sentiment['confidence'])+'")\n') 

    for keyphrase in post['keyphrase']:
        tmp_set.add('g.addV("keyphrase").property(id, "'+keyphrase['text'].lower()+'")\n')

        tmp_list.append('g.addE("relates").from(g.V("'+post['id']+'")).to(g.V("'+keyphrase['text'].lower()+'")).property("confidence", "'+str(keyphrase['confidence'])+'")\n')
    
lines.extend(list(tmp_set))
lines.extend(tmp_list)


with open('media_seed_data.txt', 'w') as f:
    f.writelines(lines)


In [92]:
from itertools import groupby

sentiment_weight = {'negative': 1.3, 'mixed': 1.2, 'positive': 1.1, 'neutral': 1.0}

def choose_sentiment(sentiment_list):

    def key_func(k):
        return k['text'].lower()

    def aggregate(obj_list):
        duration = sum(list(map(lambda obj: obj['end'] - obj['begin'], obj_list)))
        confidence = max(list(map(lambda obj: obj['confidence'], obj_list)))

        return {'duration': duration, 'confidence': confidence}

    sentiments = sorted(sentiment_list, key=key_func)
    sentiments_agg = []
    for key, values in groupby(sentiments, key_func):
        sentiment = aggregate(list(values))
        sentiment['text'] = key
        sentiments_agg.append(sentiment)

    return max(sentiments_agg, key=lambda k: k['duration'] * sentiment_weight[k['text']])

In [95]:
lines = []
tmp_set = set()
tmp_list = []

do_analysis_on = {'keyphrase_contains': 'monetisation', 'keyphrase': 'demonetisation'}

lines.append('g.addV("sentiment").property(id, "neutral")\n')
lines.append('g.addV("sentiment").property(id, "positive")\n')
lines.append('g.addV("sentiment").property(id, "negative")\n')
lines.append('g.addV("sentiment").property(id, "mixed")\n')

for post in media_data:
    lines.append('g.addV("video").property(id, "'+post['id']+'").property("title", "'+post['title']+'")\n')

    sentiment = choose_sentiment(post['sentiment'])
    lines.append('g.addE("relates").from(g.V("'+post['id']+'")).to(g.V("'+sentiment['text'].lower()+'")).property("confidence", "'+str(sentiment['confidence'])+'")\n') 

    for keyphrase in post['keyphrase']:
        keyphrase_id = keyphrase['text'].lower()
        
        if do_analysis_on['keyphrase_contains'] in keyphrase_id:
            keyphrase_id = do_analysis_on['keyphrase']
        
        tmp_set.add('g.addV("keyphrase").property(id, "'+keyphrase_id+'")\n')

        tmp_list.append('g.addE("relates").from(g.V("'+post['id']+'")).to(g.V("'+keyphrase_id+'")).property("confidence", "'+str(keyphrase['confidence'])+'")\n')
    
lines.extend(list(tmp_set))
lines.extend(tmp_list)

with open('media_seed_data.txt', 'w') as f:
    f.writelines(lines)


In [96]:
%%bash

GRAPH_NOTEBOOK_INSTALL_PATH=`pip show graph-notebook | grep "Location" | cut -d ' ' -f2`
echo $GRAPH_NOTEBOOK_INSTALL_PATH
mkdir -p $GRAPH_NOTEBOOK_INSTALL_PATH/graph_notebook/seed/queries/propertygraph/demonetisation
cp media_seed_data.txt $GRAPH_NOTEBOOK_INSTALL_PATH/graph_notebook/seed/queries/propertygraph/demonetisation/
ls -al $GRAPH_NOTEBOOK_INSTALL_PATH/graph_notebook/seed/queries/propertygraph/demonetisation

/home/ec2-user/anaconda3/envs/JupyterSystemEnv/lib/python3.6/site-packages
total 60
drwxrwxr-x 2 ec2-user ec2-user  4096 Nov 25 10:11 .
drwxrwxr-x 8 ec2-user ec2-user  4096 Nov 25 10:11 ..
-rw-rw-r-- 1 ec2-user ec2-user 49736 Nov 25 11:59 media_seed_data.txt


In [97]:
%seed --model Property_Graph --dataset demonetisation --run

Dropdown(description='Data model:', options=('', 'Property_Graph', 'RDF'), value='')

Dropdown(description='Data set:', layout=Layout(visibility='hidden'), options=(), value=None)

Button(description='Submit', layout=Layout(visibility='hidden'), style=ButtonStyle())

Output()

Output()

In [98]:
my_node_labels = '{"sentiment":"T.id","keyphrase":"T.id","video":"title"}'
my_edge_labels = '{"relates":"confidence"}'

In [99]:
%%gremlin -p v,ine,outv,oute,inv -d $my_node_labels -de $my_edge_labels

g.V().has(id, 'demonetisation').inE('relates').outV().outE('relates').inV().hasLabel('sentiment').path().by(valueMap(true))



In [70]:
%%gremlin -p v,ine,outv,oute,inv -d $my_node_labels -de $my_edge_labels

//TODO: use contains logic
g.V().has(id, TextP.containing('monetisation')).inE('relates').outV().outE('relates').inV().hasLabel('sentiment').path().by(valueMap(true))



In [91]:
%%gremlin

g.V().limit(100000).drop()